IMPORTS

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

In [ ]:
ep=pd.read_csv("expenses.csv")
print(ep.head())

EXPLORATORY DATA ANALYSIS(EDA)

In [ ]:
ep.info()

In [ ]:
ep.describe()

CHECKING MISSING VALUES

In [ ]:
ep.isnull().sum()

CALCULATE OVERALL EXPENSES

In [ ]:
#total_expenses = df['Amount'].sum()
total_expenses=ep.Amount.sum()
print(total_expenses)

EXPENSES BASED ON CATEGORIES

In [ ]:
#expense_by_category = df.groupby('Category')['Amount'].sum()
expenses_by_category=ep.groupby('Category').Amount.sum()
expenses_by_category

VISUALISATION OF expenses_by_category USING BAR CHART

In [ ]:
expenses_by_category.plot(kind='bar', color='b')
plt.title('Expenses by Category')
plt.xlabel('Category')
plt.ylabel('Amount Spent')

TIME BASED ANALYSIS

In [ ]:
ep['Date']=pd.to_datetime(ep['Date'])

DAILY SPENDING TREND

In [ ]:
daily_expense=ep.groupby('Date')['Amount'].sum()
daily_expense.plot(kind='line',marker='o',color='teal')
plt.title('Daily Expenses')
plt.xlabel('date')
plt.ylabel('expenditure')
plt.plot()


WEEKLY OR MONTHLY SPENDING TREND 

In [ ]:
ep['week']=ep['Date'].dt.isocalendar().week

weekly_expenses=ep.groupby('week')['Amount'].sum()
print(weekly_expenses)
weekly_expenses.plot(kind='line', marker='o', color='purple')
plt.title('weekly expenses')
plt.xlabel('week')
plt.ylabel('expenditure')
plt.show()

Identify High-Spending Days

In [ ]:
high_spending_days =daily_expense[daily_expense>100]
high_spending_days

ANALYZING CATEGORIES WITH HIGHEST SPENDING IMPACT

In [ ]:
sorted_expenses_by_category=expenses_by_category.sort_values(ascending=False)
sorted_expenses_by_category.plot(kind='pie',autopct='%1.1f%%', startangle=140)
plt.title('Percentage of Total Expenses by Category')
plt.ylabel('')
sorted_expenses_by_category

AVERAGE DAILY SPENDING

In [ ]:
mean_daily_expense=daily_expense.mean()
print(mean_daily_expense)

SEPERATE TARGET FROM PREDICTORS

In [ ]:
target=daily_expense
ep_grouped = ep.groupby('Date').sum()
ep_grouped = ep_grouped.drop(['Amount'], axis=1)
ep_=ep_grouped

SPLITTING THE DATA FOR TRAINING AND VALIDATION

In [ ]:
train_full_ep, valid_full_ep, train_target, valid_target = train_test_split(ep_, target, train_size=0.8, test_size=0.2, random_state=42, shuffle=False)

IDENTIFYING NUMERICAL AND CATEGORICAL COLUNMS

In [ ]:
numerical_cols=[cname for cname in train_full_ep.columns if train_full_ep[cname].dtype in ['int64', 'float64']]
categorical_cols=[cname for cname in train_full_ep.columns if train_full_ep[cname].dtype =='object']

KEEP SELECTED COLUMNS ONLY

In [ ]:
selected_cols=numerical_cols+categorical_cols
train_ep=train_full_ep[selected_cols].copy()
valid_ep=valid_full_ep[selected_cols].copy()

OneHotEncoding THE CATEGORICAL COLUMNS

In [ ]:
numerical_transformer=SimpleImputer(strategy='median')

categorical_transformer=Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

BUNDLING THE NUMERICAL AND CATEGORICAL COLUMNS

In [ ]:
preprocessor=ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

BUNDLING THE PREPROCESSOR AND MODEL INTO A PIPELINE 

In [ ]:
ep_pipeline=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42, max_depth=5)),
])

FITTING THE MODEL AND GETTING PREDICTION ACCORING TO THE TOTAL DAILY EXPENDITURE

In [ ]:
ep_pipeline.fit(train_ep, train_target)
preds=ep_pipeline.predict(valid_ep)

score=mean_absolute_error(valid_target,preds)
print("MAE=",score)

COMPARISION OF ACTUAL AND PREDICTED VALUES

In [ ]:
comparison_df = pd.DataFrame({
    'Actual': valid_target.values,
    'Predicted': preds
}, index=valid_target.index)

print(comparison_df)

VISUALIZING THE PRDICTIONS TO THE ACTUAL VALUES

In [ ]:
preds_series = pd.Series(preds, index=valid_target.index)
plt.figure(figsize=(10,6))
plt.plot(valid_target.index,valid_target,label='Actual')
plt.plot(valid_target.index, preds_series, label='Predicted')
plt.title('Actual VS Predicted')
plt.xlabel('Date')
plt.ylabel('Amount')

plt.legend()
plt.show()